In [1]:
from torch.utils.data import Dataset
import glob
import os
from skimage.io import imread
import numpy as np
from ipynb.fs.full.functools import Cases, Data

from tqdm import tqdm

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
device      = torch.device('cpu') 
num_workers = 0
image_size  = 224 
batch_size  = 4

In [3]:
cases = Cases()
data = cases.get_array()

In [4]:
augs = A.Compose([A.Resize(height = image_size, 
                           width  = image_size),
                  A.Normalize(mean = (0, 0, 0),
                              std  = (1, 1, 1)),
                  ToTensorV2()])

In [5]:
# dataset
image_dataset = Data(data=data, transform=augs)

image_loader = DataLoader(image_dataset, 
                          batch_size  = batch_size, 
                          shuffle     = False, 
                          num_workers = 0,
                          pin_memory  = True)

In [6]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(image_loader):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])
    
# pixel count
count = len(image_dataset) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

100%|█████████████████████████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 146.68it/s]


mean: tensor([0.4247, 0.4662, 0.6567])
std:  tensor([0.1296, 0.1301, 0.1386])
